#### 감정 분석 자연어 처리 
1. data 폴더 안에 ratings_train.txt 파일을 로드 
2. 데이터를 상위 500개 데이터만 추출
3. 리뷰 데이터와 감정 데이터로 나눠준다. 
4. 리뷰 데이터를 토큰화(komoran함수 이용)
5. Word2Vec 학습
    - window -> 3
    - epochs -> 5
    - min_count -> 5
    - sg -> 1
    - seed -> 42
6. 벡터화(Word2Vec, 단위 벡터의 평균)
7. 분류 모델 ( SVC, Logistic )
8. train, test을 이용하여 2개의 모델중 성능이 높은 모델이 무엇인가?
9. 단위 백터의 평균의 성능과 단위 벡터 + 중요도 평균의 성능의 차이를 확인
